# F2Heal - PAC analysis



## Intro

In this notbook we try to reproduce the results of Swan et al. in "Elevated Synchrony in Parkinson's Disease Detected with
Electroencephalography" using Python.

The following domain specific Python libraries are used:
* [MNE-Python](https://mne.tools/)
* [pactools](https://github.com/pactools/pactools)
* [openneuro-py](https://pypi.org/project/openneuro-py/)

The following general purpose libraries are used:
* [NumPy](https://numpy.org/)
* [MatplotLib](https://matplotlib.org/)

We will use the San Diego dataset [published on OpenNeuro](https://openneuro.org/datasets/ds002778)

## Preparation


Install required python packages and configure MatplotLib output for this notebook evironment:

In [ ]:
%pip install mne mne_bids pactools openneuro-py autoreject ipympl
%matplotlib widget

## Pyhton script

### Setting the scene

As usual, we like it open source!

In [ ]:
# SPDX-License-Identifier: AGPL-3.0-or-later 


The Python script starts by importing all required libraries

In [ ]:
import os
import mne
import mne_bids
import openneuro

import autoreject

import numpy as np
import pactools
import matplotlib.pyplot as plt

We introduce two configurable parameters:
* `verbose` can be set to **False** to reduce command output

In [ ]:
verbose = True

* `n_jobs` controls the number of parallel threads and should be set to to the number of CPU cores available in your environment for optimal performance

In [ ]:
n_jobs = 2

We'll define a class with configuration data describing the OpenNeuro dataset that will be used from here on.

In [ ]:
class ONDataset:
    """
    Open EEG dataset from OpenNeuro
    Parkinson's EEG dataset <https://openneuro.org/datasets/ds002778
    """
    dataset = 'ds002778'
    bids_root = "/tmp/mne"
    datatype = 'eeg'
    task = 'rest'
    suffix = 'eeg'


Next, we'll also define a class with configuration parameters relevant for the PAC generation. These variables can be modified.

The variable `low_fq_range` defines the frequency range for which the phase is extracted and will become the X-axis of the comodulogram.

The variable `high_fq_range` defines the frequency range for which the amplitude component is extracted and will become the Y-axis of the comodulogram.

The syntax of the Numpy linspace function is: `np.linspace(<lower freq in Hz>, <upper freq in Hz>, <number of intervals>)`. The `<number of intervals>` determine the resolution of the comodulogram.

The variable `method` specifies the PAC calculation method. Valid methods are:  `'tort', 'penny', 'vanwijk', 'duprelatour', 'colgin', 'sigl', 'bispectrum'`

Swan et al. have used the equivalent of the `'tort'` method in MATLAB. This method does not produce similar results in MNE-Python. The method `'duprelatour'` does. See [this Github issue](https://github.com/pactools/pactools/issues/33) for more details.

In [ ]:
class PACSettings:
    low_fq_range = np.linspace(13,50,150)
    low_fq_width = 2
    high_fq_range= np.linspace(50,200,150)
    high_fq_width= 4
    method= 'duprelatour'

    def create_Comodulogram(fs):
        return pactools.Comodulogram (fs=fs,
                                      low_fq_range=PACSettings.low_fq_range,
                                      low_fq_width=PACSettings.low_fq_width,
                                      high_fq_range=PACSettings.high_fq_range,
                                      high_fq_width=PACSettings.high_fq_width,
                                      method=PACSettings.method,
                                      progress_bar=verbose,
                                      n_jobs=n_jobs)

### Loading the OpenNeuro data

We need to pick a valid subject from the OpenNeuro dataset. You can pick an ID from: 3, 5, 6, 9, 11, 12, 13, 14, 16, 17, 19, 22, 23, 26, 28

In [ ]:
subject = "pd6"

Next, we'll download the relevant EEG session for this subject from OpenNeuro. You might need to restart this step a second time if it fails at first.

In [ ]:
def fetch_raw(subject, session):

    if not os.path.isdir(ONDataset.bids_root):
        os.makedirs(ONDataset.bids_root)

    openneuro.download(dataset=ONDataset.dataset, target_dir=ONDataset.bids_root, include=[f"sub-{subject}"])
    bids_path = mne_bids.BIDSPath(root=ONDataset.bids_root,
                         task=ONDataset.task, suffix=ONDataset.suffix,
                         datatype=ONDataset.datatype, session=session,
                         subject=subject )

    raw = mne.io.read_raw_bdf(bids_path, verbose=verbose, preload=True)

    return raw

raw_on = fetch_raw(subject, 'on')

### Data inspection (optional)

This data has now been loaded in a MNE-Python `raw` structure, that we can inspect:

In [ ]:
print(raw_on.info)
print(raw_on.info["subject_info"])

We can also visulaize the raw stream:

In [ ]:
_ = raw_on.plot(duration=5, n_channels=30)

### Preprocessing 1/2

Drop the external channels from the stream

In [ ]:
_ = raw_on.drop_channels(['EXG1', 'EXG2', 'EXG3', 'EXG4','EXG5', 'EXG6', 'EXG7', 'EXG8', 'Status'])

Set the correct headset montage details, specify the model mentioned by George et al. in "Dopaminergic therapy in Parkinson's disease decreases cortical beta band coherence in the resting state and increases cortical beta band power during executive control"

In [ ]:
_ = raw_on.set_montage("biosemi32", on_missing="ignore", verbose=verbose)

Create a new virtual electrode by averaging all channels, the re-reference the data according to this new virtual reference.

In [ ]:
_ = raw_on.set_eeg_reference(ref_channels='average',verbose=verbose)

Apply a high-pass filter with 0.5Hz cut-off frequency

In [ ]:
_ = raw_on.filter(0.5, None, fir_design='firwin', phase='zero-double', n_jobs=n_jobs, verbose=verbose)

#### Visualizing PSD (optional)

Now that montage data has been added, we can visualize the Power Spectral Density of the `raw_on` stream

In [ ]:
_ = raw_on.compute_psd(fmax=50).plot(picks="data", exclude="bads", amplitude=False)

### Preprocessing 2/2

Divide the current raw stream in equal epochs of 3sec length.

In [ ]:
epochs_all_on = mne.make_fixed_length_epochs(raw_on, duration=3, preload=True, verbose=verbose)

Apply the default autoreject filter

In [ ]:
ar = autoreject.AutoReject(n_interpolate=[1, 2, 3, 4], random_state=11, n_jobs=n_jobs, verbose=verbose)
ar.fit(epochs_all_on)
epochs_on = ar.transform(epochs_all_on)

### Visualize autoreject heatmap (optional)

We can visualize the actions of autoreject for channels 'C3' and 'C4'. 

Due to an [issue in pactools](https://github.com/autoreject/autoreject/issues/226) I use a hack to visualize only 'C3' and 'C4' via ar.picks_, resulting in a messy graf.

In [ ]:
ar.picks_ = [ i for i in range(len(ar.get_reject_log(epochs_all_on).ch_names)) if ar.get_reject_log(epochs_all_on).ch_names[i] in ['C3','C4']  ]
_ = ar.get_reject_log(epochs_on, picks=['C3', 'C4']).plot('horizontal')


### Calculating PAC

Create a 3-element array for holding the Comodulogram objects, for electrode C3, C4 and their average respectively.

In [ ]:
pacs_on = [ PACSettings.create_Comodulogram(raw_on.info['sfreq']) for _ in range(3)]

Fit C3 data to the first element of the array.

In [ ]:
pacs_on[0].fit(epochs_on.get_data(picks=['C3'])[:,0,:])

C4 data to the second element of the array

In [ ]:
pacs_on[1].fit(epochs_on.get_data(picks=['C4'])[:,0,:])

Average of both in the third element.

In [ ]:
pacs_on[2].comod_ = np.mean([pacs_on[0].comod_, pacs_on[1].comod_],axis=0)

Now that we have calculated the results for on-session, repeat for off-session.

In [ ]:
raw_off = fetch_raw(subject, 'off')

raw_off.drop_channels(['EXG1', 'EXG2', 'EXG3', 'EXG4','EXG5', 'EXG6', 'EXG7', 'EXG8', 'Status'])
raw_off.set_montage("biosemi32", verbose=verbose)
raw_off.set_eeg_reference(ref_channels='average',verbose=verbose)
raw_off.filter(0.5, None, fir_design='firwin', phase='zero-double', n_jobs=n_jobs, verbose=verbose)

epochs_all_off = mne.make_fixed_length_epochs(raw_off, duration=3, preload=True, verbose=verbose)
ar = autoreject.AutoReject(n_interpolate=[1, 2, 3, 4], random_state=11, n_jobs=n_jobs, verbose=verbose)
ar.fit(epochs_all_off)
epochs_off = ar.transform(epochs_all_off)

Combine the OFF results in another 3-element array.

In [ ]:
pacs_off = [ PACSettings.create_Comodulogram(raw_off.info['sfreq']) for _ in range(3)]

pacs_off[0].fit(epochs_off.get_data(picks=['C3'])[:,0,:])
pacs_off[1].fit(epochs_off.get_data(picks=['C4'])[:,0,:])
pacs_off[2].comod_ = np.mean([pacs_off[0].comod_, pacs_off[1].comod_],axis=0)

To set a common scale for the Comodulograms, we need to find the Vmax in all results.

In [ ]:
vmax = max([pac.comod_.max() for pac in np.concatenate((pacs_on, pacs_off))])

Calculate the Modulation Index (MI) for every PAC by averaging the results.

In [ ]:
mis_on  = [ np.mean(pac.comod_) for pac in pacs_on ]
mis_off = [ np.mean(pac.comod_) for pac in pacs_off ]

### Results

Print the MI values

In [ ]:
print("%s " % (subject))
print(" ON  C3:%.3E C4:%.3E AVG:%.3E" % (mis_on[0], mis_on[1], mis_on[2]))
print(" OFF C3:%.3E C4:%.3E AVG:%.3E" % (mis_off[0], mis_off[1], mis_off[2]))

Create Matplotlib object with the desired structure and size and plot the previously calculated results to it:

In [ ]:
fig,axs=plt.subplots(2,3,figsize=(14,7))

_ = plt.suptitle("Subject: %s  PAC Method: %s"
             % (subject, PACSettings.method ))

pacs_on[0].plot(titles=["ON C3 MI=%.2E"  % mis_on[0]], axs=[axs[0,0]], vmin=0, vmax=vmax)
pacs_on[1].plot(titles=["ON C4 MI=%.2E"  % mis_on[1]], axs=[axs[0,1]], vmin=0, vmax=vmax)
pacs_on[2].plot(titles=["ON AVG MI=%.2E" % mis_on[2]], axs=[axs[0,2]], vmin=0, vmax=vmax)

pacs_off[0].plot(titles=["OFF C3 MI=%.2E"  % mis_off[0]], axs=[axs[1,0]], vmin=0, vmax=vmax)
pacs_off[1].plot(titles=["OFF C4 MI=%.2E"  % mis_off[1]], axs=[axs[1,1]], vmin=0, vmax=vmax)
pacs_off[2].plot(titles=["OFF AVG MI=%.2E" % mis_off[2]], axs=[axs[1,2]], vmin=0, vmax=vmax)
plt.show()
